This code processes a set of images and then returns the average of the images 
and first set of principal components

In [ ]:
#Calculates the number of vectors needed in PCA to explain 95% of the variation
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1
print(d)

In [ ]:
from PIL import Image
from numpy import *
from pylab import *
from sklearn.decomposition import PCA
import os

#Returns the files in the filepath
def getim_list(path):
    return[os.path.join(path,f) for f in os.listdir(path) if f.endswith('jpg')]

#filepath to where the images are stored
pathlist='filepath'

imlist=getim_list(pathlist)

#Removes the smaller images where there is no streetview image
imlist2=[]
for im in imlist:
    if os.stat(im).st_size> 8923:
     imlist2.append(im)

#Loops through the first 1000 images and flattens them into an array
#The convert('L') switches it into black and white
immatrix=array([array(Image.open(im).convert('L')).flatten() for im in imlist2[1:1000]], 'f') 

############################################################################################
#The pca function
def pca(X):
# Principal Component Analysis
# input: X, matrix with training data stored as flattened arrays in rows
# return: projection matrix (with important dimensions first), variance and mean.

# get dimensions
 num_data,dim = X.shape
# center data
 mean_X = X.mean(axis=0)
 X = X - mean_X
 if dim>num_data:
 # PCA - compact trick used
   M = dot(X,X.T) # covariance matrix
   e,EV = linalg.eigh(M) # eigenvalues and eigenvectors
   tmp = dot(X.T,EV).T # this is the compact trick
   V = tmp[::-1] # reverse since last eigenvectors are the ones we want
   S = sqrt(e)[::-1] # reverse since eigenvalues are in increasing order
   for i in range(V.shape[1]):
     V[:,i] /= S
 else:
# PCA - SVD used
  U,S,V = linalg.svd(X)
  V = V[:num_data] # only makes sense to return the first num_data
# return the projection matrix, the variance and the mean
 return V,S,mean_X

################################################################################################
#Gets the size of the first image
im=array(Image.open(imlist[0]))
m,n=im.shape[0:2]

#Calculates the pca analysis                
V,S,immean=pca(immatrix)

figure()
gray()
#sublot five rows, four columns and the first picture which is the average image
subplot(5,4,1)
#Converts the image back to picture shape and plots the mean picture
imshow(immean.reshape(m,n))

for i in range(19):
    subplot(5,4,i+2)
    imshow(V[i].reshape(m,n))

show()
